In [ ]:
pip install torch torchvision


In [ ]:
pip install gdown


In [ ]:
# Step 1: Download data from Google Drive
train_url = 'https://drive.google.com/uc?id=15v2CmnA-BqLMU0E86whGEnSl6Wbv4mKW'
val_url = 'https://drive.google.com/uc?id=1uSswsVLN1ASLWd4KSwocYvGaE0rgXv9l'

gdown.download(train_url, output='train.zip', quiet=False)
gdown.download(val_url, output='val.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=15v2CmnA-BqLMU0E86whGEnSl6Wbv4mKW
To: /content/train.zip
100%|██████████| 665k/665k [00:00<00:00, 74.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uSswsVLN1ASLWd4KSwocYvGaE0rgXv9l
To: /content/val.zip
100%|██████████| 241k/241k [00:00<00:00, 75.6MB/s]


'val.zip'

In [ ]:

# Step 2: Extract ZIP files
with zipfile.ZipFile('train.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset/train')

with zipfile.ZipFile('val.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset/val')

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])

In [ ]:
# Step 4: Load datasets
train_dataset = datasets.ImageFolder('dataset/train', transform=transform)
val_dataset = datasets.ImageFolder('dataset/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
# Step 5: Load pre-trained AlexNet
from torchvision.models import alexnet, AlexNet_Weights
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)

# Freeze all feature extractor layers
for param in model.parameters():
    param.requires_grad = False

# Modify classifier to match number of classes
num_classes = len(train_dataset.classes)
model.classifier[6] = nn.Linear(4096, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
# Step 6: Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.classifier.parameters(), lr=0.001)

In [ ]:
# Step 7: Training loop
epochs = 15

print("\n Starting Training...\n")
for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss:.4f}, Training Accuracy: {acc:.2f}%")


 Starting Training...

Epoch [1/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [2/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [3/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [4/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [5/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [6/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [7/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [8/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [9/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [10/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [11/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [12/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [13/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [14/15], Loss: 0.0000, Training Accuracy: 100.00%
Epoch [15/15], Loss: 0.0000, Training Accuracy: 100.00%


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
print(f"\n Validation Accuracy: {val_acc:.2f}%")


 Validation Accuracy: 100.00%
